## Part 5
### Implementation of a second order HMM and decoding with Viterbi

In [71]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        return train_list
    
train_list = load_original_train()

In [72]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 24714, 'I-positive': 227, 'B-positive': 809, 'I-negative': 236, 'B-negative': 705, 'I-neutral': 44, 'B-neutral': 119}


In [73]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
    
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences

modified_train_list = load_modified_train_file()   
dev_in_list = load_files()
sentences = split_into_sentences(dev_in_list)

In [74]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    

def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = annotationDict[word_tag_pair_list[len(word_tag_pair_list)-1]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict

def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict

def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[len(each_word)-1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber

modifiedWordDict = get_word_dict()
wordTagPairDict = count_word_tag_pair_dict()
giantEmissionDict = store_estimate_emission_fix()
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
giantTransitionDict = store_estimate_transition()

In [75]:
def store_second_order_transition():
    transitionCount = {}
    startStateCount = {}
    giantSecondTransitionDict = {}        
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if startStateCount.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                startStateCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                startStateCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
                
        for i in range(len(taggedEachTweet) - 2):
            if transitionCount.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1] + " " + taggedEachTweet[i+2]) == None:
                transitionCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]+ " "+ taggedEachTweet[i+2]] = 1
            else:
                transitionCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]+ " "+ taggedEachTweet[i+2]] += 1
    for eachTransition in transitionCount:
        eachTransitionList = eachTransition.split(" ") #START O O
        startState = eachTransitionList[0] + " " + eachTransitionList[1] # START O
        countTop = transitionCount[eachTransition]
        countBottom = startStateCount[startState]
        transitionParam = countTop/countBottom
        giantSecondTransitionDict[eachTransition] = transitionParam

    return giantSecondTransitionDict

giantSecondTransitionDict = store_second_order_transition()
print(giantSecondTransitionDict)

{'START O O': 0.7865397688647179, 'O O O': 0.8778399888945444, 'O O B-negative': 0.02392300217481838, 'O B-negative O': 0.8264331210191083, 'B-negative O O': 0.929553264604811, 'O O B-positive': 0.0251260931932812, 'O B-positive O': 0.8502024291497976, 'B-positive O O': 0.9089595375722543, 'O O STOP': 0.06996436999676091, 'START B-negative O': 0.8181818181818182, 'START O B-positive': 0.11964649898028552, 'O B-positive I-positive': 0.145748987854251, 'B-positive I-positive I-positive': 0.6460176991150443, 'I-positive I-positive O': 0.6460176991150443, 'I-positive O O': 0.8125, 'START B-neutral O': 0.875, 'B-neutral O O': 0.9381443298969072, 'START O B-negative': 0.06866077498300475, 'O B-negative I-negative': 0.16560509554140126, 'B-negative I-negative I-negative': 0.635593220338983, 'I-negative I-negative I-negative': 0.3644067796610169, 'I-negative I-negative O': 0.635593220338983, 'I-negative O O': 0.9658119658119658, 'O O B-neutral': 0.003146545740595067, 'O B-neutral O': 0.8058252

In [76]:
def backtrack_second_order(piList, sentence):
    path = []
    singleStringToBeWritten = ""
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
        
    if len(piList) == len(sentence):
        for i in range(len(piList)-1):
            if i == 0:
                lastLayer = piList[len(piList)- (i+1)]
                maxPiTag = max(lastLayer, key=lastLayer.get)
                maxPiTagList = maxPiTag.split(" ")
                path.append(maxPiTagList[len(maxPiTagList)-1])
                path.append(maxPiTagList[0])

            else:
                currentLayer = piList[len(piList) - (i+1)] #2nd last layer onwards
                targetedTag = path[i-1]
                scoreDict = {}
                for tags in currentLayer:
                    if giantSecondTransitionDict.get(tags + " " + targetedTag) != None:
                        scoreOfTag = currentLayer[tags]*giantSecondTransitionDict[tags + " " + targetedTag]
                        scoreDict[tags] = scoreOfTag
                bestScoreTag = max(scoreDict, key=scoreDict.get)
                bestScoreTagList = bestScoreTag.split(" ")
                path.append(bestScoreTagList[0])
                if bestScoreTagList[0] == "START":
                    path.append(bestScoreTagList[len(bestScoreTagList)-1])
    path.remove("STOP")
    path.remove("START")
    for j in range(len(sentence)):
        if sentence[j] == "":
            singleStringToBeWritten += "\n"
        else:
            singleStringToBeWritten += sentence[j] + " " + path[len(path)-(j+1)] + "\n"
    return singleStringToBeWritten
piList = [{'O': 0.10952242946302841, 'B-negative': 0.0019577571933592874, 'B-positive': 0.010624504726325918, 'B-neutral': 0.0009040105193951349}, {'START O': 0.00024221306515862052, 'START B-negative': 0.0001000958565025801, 'START B-positive': 0.0012160526263888527, 'START B-neutral': 1.9652402595546413e-05}, {'O B-negative': 2.5578783356529717e-08, 'O B-positive': 9.94562791068425e-08}, {'B-positive STOP': 1.413233095656732e-09}]
sentence = ['AVOID', 'THAT', 'PLACE', '']
piList2 = [{'B-positive': 0.00014504443312390332, 'B-neutral': 8.21827744904668e-05}, {'START B-positive': 1.701560172955939e-06}, {'B-positive O': 7.4384900942866305e-09}, {'O O': 2.2814373537614629e-10}, {'O O': 1.3952466906739993e-13}, {'O O': 2.5096579686371738e-17}, {'O O': 1.375919532845953e-18}, {'O STOP': 1.0482944960703226e-19}]
sentence2 = ['lobster', 'was', 'good', ',', 'nothing', 'spectacular', '.', '']
# backtrack_second_order(piList, sentence)
# backtrack_second_order(piList2, sentence2)

In [77]:
def second_order_viterbi(sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    sentence.append("") #so we can detect the end
    piList = []
    if len(sentence) == 2:
        resultToBeReturned = sentence[0] + " " + "O"
        return resultToBeReturned
    for i in range(len(sentence)):
        word = sentence[i]
        if word in modifiedWordDict:
            word = word
        else:
            word = "#UNK#"
            
        piLayer = {}
        
        if i == 0:
            for tag in all_tags:
                if giantTransitionDict.get("START" + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                    pi = giantTransitionDict["START" + " " + tag]*giantEmissionDict[word + " " + tag]
                    piLayer[tag] = pi
            piList.append(piLayer)
        
        elif i == 1:
            previousLayer = piList[len(piList)-1]
            for tag in all_tags:
                tempLayer = {}
                for previous_tag in previousLayer:
                    if giantSecondTransitionDict.get("START" + " " + previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                        pi = previousLayer[previous_tag]*giantSecondTransitionDict["START" + " " + previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
                        tempLayer["START" + " " + previous_tag] = pi
                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get) 
                    maxPiScore = max(tempLayer.values())
                    piLayer[maxPiTag] = maxPiScore
            piList.append(piLayer)
            
        else:
            
            previousLayer = piList[len(piList)-1]
#             print(previousLayer)
            if word == "":
#                 print(piList)
                tempLayer = {}
                for previous_tags in previousLayer:
                    previous_tags_list = previous_tags.split(" ")
                    if giantSecondTransitionDict.get(previous_tags + " " + "STOP") != None:
                        pi = previousLayer[previous_tags]*giantSecondTransitionDict[previous_tags + " " + "STOP"]
                        tagOneLayerBefore = previous_tags_list[len(previous_tags_list)-1]
                        tempLayer[tagOneLayerBefore+ " " + "STOP"] = pi
                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get) 
                    maxPiScore = max(tempLayer.values())
                    piLayer[maxPiTag] = maxPiScore
#                 else:
#                     piLayer[]
                piList.append(piLayer)
                singleStringToBeWritten = backtrack_second_order(piList, sentence)
#                 print(singleStringToBeWritten)
                return singleStringToBeWritten
            else:
                for tag in all_tags:
                    tempLayer = {}
                    for previous_tags in previousLayer:
                        previous_tags_list = previous_tags.split(" ")
                        if giantSecondTransitionDict.get(previous_tags + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                            pi = previousLayer[previous_tags]*giantSecondTransitionDict[previous_tags + " " + tag]*giantEmissionDict[word + " " + tag]
                            tagOneLayerBefore = previous_tags_list[len(previous_tags_list)-1]
                            tempLayer[tagOneLayerBefore+ " " + tag] = pi
#                         else:
#                             print("problem from %s  to %s" % (previous_tags, tag))
#                             print(word)
#                             print(giantSecondTransitionDict.get(previous_tags + " " + tag))
#                             # riz only has emission from B - positive
#                             # no transition from I-neutral O to B-positive
#                             print(giantSecondTransitionDict)
#                             print(giantEmissionDict.get(word + " " + tag))
                    if len(tempLayer) != 0:
                        maxPiTag = max(tempLayer, key=tempLayer.get) 
                        maxPiScore = max(tempLayer.values())
                        piLayer[maxPiTag] = maxPiScore
                    
                # any other way to fix this??
                if len(piLayer) == 0:
                    for previous_tags in previousLayer:
                        previous_tags_list = previous_tags.split(" ")

                        piLayer[previous_tags_list[len(previous_tags_list)-1] + " " + "O"] = 0

                piList.append(piLayer)
    
                        

            
giantStringToBeWritten = ""    
for sentence in sentences:
    singleStringToBeWritten = second_order_viterbi(sentence)
    print(singleStringToBeWritten)
    giantStringToBeWritten += singleStringToBeWritten
print(giantStringToBeWritten)
f = open("dev.p5.out","w+")
f.write(giantStringToBeWritten)
# second_order_viterbi(['des', 'plats', 'sans', 'passion', ',', 'vaisselle', 'cassée', ',', 'plat', 'du', 'jour', '(', 'riz', 'crustacés', 'en', 'gros', ')', 'très', 'sec', ',', 'sans', 'sauce', ',', 'gambas', 'pas', 'fraîches', ',', 'une', 'pauvre', 'décoration', 'superflue', '...', 'bref', 'aucun', 'goût', '.'])
# second_order_viterbi(["lobster", "was", "good", ",", "nothing","spectacular","."])

La O
décoration O
est O
horrible O
. O


Assiettes O
de O
tapas B-positive
ridicules O
. O


Nous O
avons O
été O
attiré O
comme O
les O
autres O
personnes O
par O
la O
terrasse B-positive
et O
les O
prix O
bas O
. O


Le O
plat O
du O
jour O
comme O
son O
nom O
l'indique O
est O
plutôt O
un O
gage O
de O
fraicheur O
et O
qualité O
; O
ce O
jour O
là O
si O
le O
sanglier O
était O
convenable O
, O
l O
' O
accompagnement O
très O
limite O
à O
savoir O
des O
pâtes B-negative
desséchées O
et O
froides O
. O


Très O
bon O
bistrot B-positive
. O


Des O
pizzas O
locales O
que O
vous O
ne O
mangerez O
pas O
ailleurs O
, O
accommodant O
les O
spécialités O
du O
coin O
. O


Ensuite O
les O
plats B-positive
... O
pas O
de O
doutes O
la O
spécialité O
très O
maîtrisé O
est O
le O
burger O
maison O
! O


Reste O
le O
service B-positive
, O
sympathique O
mais O
un O
peu O
trop O
décontracté O
qui O
ne O
suffit O
pas O
à O
relever O
l'absence O
de O
qualité O
de O
ce O
restaurant O
. O


En O
bre

27727